# My Kannada MNIST Kaggle competition
### with Fastai v1

In [ ]:
from fastai.vision import *

In [ ]:
!ls ../input/Kannada-MNIST

In [ ]:
path = Path('../input/Kannada-MNIST')
train_csv = path/'train.csv'

In [ ]:
data = pd.read_csv(train_csv)

In [ ]:
data.head()

In [ ]:
y = data.label.values

In [ ]:
X = torch.tensor(data.drop('label', axis = 1).values)

In [ ]:
X[0].shape

In [ ]:
tfms = get_transforms(do_flip=False)

### Randomly Split onto Training and Validation sets

In [ ]:
rand_idx = torch.randperm(X.shape[0])
split_ratio = 0.8
split = int(X.shape[0] * split_ratio)
train_idxs = rand_idx[:split]
test_idxs  = rand_idx[split:]

X_train = X[train_idxs]
X_valid = X[test_idxs]
y_train = y[train_idxs]
y_valid = y[test_idxs]
X_train.shape, X_valid.shape

In [ ]:
def tensor2Images(x):
    return [Image(x[i].reshape(-1,28,28).repeat(3, 1, 1)/255.) for i in range(x.shape[0])]

In [ ]:
class MNISTImageList(ImageList):
    "`ImageList` of Images stored as in `items` as tensor."

    def open(self, fn):
        "No file associated to open"
        pass

    def get(self, i):
        res = self.items[i]
        self.sizes[i] = sys.getsizeof(res)
        return res

#### Get LabelLists

In [ ]:
til = MNISTImageList(tensor2Images(X_train))

In [ ]:
til[0]

In [ ]:
train_ll = LabelList(MNISTImageList(tensor2Images(X_train)),CategoryList(y_train, ['0','1','2','3','4','5','6','7','8','9']))
valid_ll = LabelList(MNISTImageList(tensor2Images(X_valid)),CategoryList(y_valid, ['0','1','2','3','4','5','6','7','8','9']))

In [ ]:
valid_ll[1][0]

In [ ]:
 valid_ll[1][1]

In [ ]:
ll = LabelLists('.',train_ll,valid_ll)

In [ ]:
data.head()

In [ ]:
test_csv  = path/'test.csv'
data = pd.read_csv(test_csv)
Xtest = torch.tensor(data.drop('id', axis = 1).values)
test_il = ItemList(tensor2Images(Xtest))

In [ ]:
ll.add_test(test_il)

In [ ]:
assert len(ll.train.x)==len(ll.train.y)
assert len(ll.valid.x)==len(ll.valid.y)

In [ ]:
ll.train.x[0]

#### Get ImageDataBunch from LabelLists

In [ ]:
dbch = ImageDataBunch.create_from_ll(ll)

In [ ]:
dbch.sanity_check()

In [ ]:
dbch

In [ ]:
dbch.show_batch(rows=4, figsize=(6,6))

# Training

In [ ]:
path = '/kaggle/working/'

In [ ]:
learn = cnn_learner(dbch,models.resnet50,metrics=accuracy, pretrained=True)

In [ ]:
learn.freeze()

In [ ]:
learn.lr_find()
learn.recorder.plot()

In [ ]:
learn.fit_one_cycle(10,slice(2e-3,2e-2))

In [ ]:
learn.recorder.plot_losses()

In [ ]:
learn.save('stage1')

In [ ]:
learn.unfreeze()

In [ ]:
learn.lr_find(start_lr=1e-9)
learn.recorder.plot()

In [ ]:
learn.fit_one_cycle(10,slice(2e-7,2e-6))

In [ ]:
learn.recorder.plot_losses()

In [ ]:
learn.show_results(ds_type=DatasetType.Train, rows=4, figsize=(8,10))

In [ ]:
learn.fit_one_cycle(5,max_lr=1e-6)

In [ ]:
learn.summary()

In [ ]:
learn.export()

# Inference

In [ ]:
preds,y = learn.get_preds(ds_type=DatasetType.Test)

In [ ]:
y = preds.argmax(dim=1)

In [ ]:
assert len(y)==len(test_il)

In [ ]:
res = pd.DataFrame(y,columns=['label'],index=range(1, 5001))
res.index.name = 'id'

In [ ]:
res.to_csv('submission.csv')